<a href="https://colab.research.google.com/github/stephanny-soares/sentiment-analysis-project/blob/Data-Science/SentimentAPI_modelagem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TEXTE API KaGGLE

In [1]:
# Instala a biblioteca Kaggle
!pip install -q kaggle
import os

# 1. Configurar a Chave e o Usuário (Substitua pelos seus dados)
# NOTA: O nome de usuário do Kaggle pode ser encontrado no seu perfil.
# A chave de API (key) é a chave que você copiou após gerar o token.

KAGGLE_USERNAME = "emanuel"  # Ex: "emanuel" (conforme a imagem)
KAGGLE_KEY = "KGAT_61c94f10a87d87f193655a905521899b"              # Ex: "7a29e92d8f94..." (A chave que você copiou)

# 2. Criar a estrutura de diretórios e o arquivo kaggle.json

# Cria a pasta .kaggle (se não existir)
!mkdir -p ~/.kaggle

# Conteúdo do arquivo kaggle.json no formato JSON
kaggle_json_content = f'{{"username":"{KAGGLE_USERNAME}", "key":"{KAGGLE_KEY}"}}'

# Escreve o conteúdo no arquivo kaggle.json
with open(os.path.expanduser('~/.kaggle/kaggle.json'), 'w') as f:
    f.write(kaggle_json_content)

# 3. Garante que o arquivo tenha as permissões corretas
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d olistbr/brazilian-ecommerce

Dataset URL: https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce
License(s): CC-BY-NC-SA-4.0
  0% 0.00/42.6M [00:00<?, ?B/s]
100% 42.6M/42.6M [00:00<00:00, 516MB/s]


In [2]:
import zipfile
import pandas as pd
import os

zip_file_name = 'brazilian-ecommerce.zip'
extracted_folder = 'olist_data'
os.makedirs(extracted_folder, exist_ok=True)

# Descompacta o arquivo principal
with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    print(f"Descompactando {zip_file_name}...")
    zip_ref.extractall(extracted_folder)

print("Arquivos CSV descompactados com sucesso.")

# Exemplo de carregamento de um arquivo:
file_to_load = os.path.join(extracted_folder, 'olist_order_reviews_dataset.csv')
df_orders = pd.read_csv(file_to_load)
print("\nPrimeiras linhas do dataset de Pedidos (Orders):")
print(df_orders.head())

Descompactando brazilian-ecommerce.zip...
Arquivos CSV descompactados com sucesso.

Primeiras linhas do dataset de Pedidos (Orders):
                          review_id                          order_id  \
0  7bc2406110b926393aa56f80a40eba40  73fc7af87114b39712e6da79b0a377eb   
1  80e641a11e56f04c1ad469d5645fdfde  a548910a1c6147796b98fdf73dbeba33   
2  228ce5500dc1d8e020d8d1322874b6f0  f9e4b658b201a9f2ecdecbb34bed034b   
3  e64fb393e7b32834bb789ff8bb30750e  658677c97b385a9be170737859d3511b   
4  f7c4243c7fe1938f181bec41a392bdeb  8e6bfb81e283fa7e4f11123a3fb894f1   

   review_score review_comment_title  \
0             4                  NaN   
1             5                  NaN   
2             5                  NaN   
3             5                  NaN   
4             5                  NaN   

                              review_comment_message review_creation_date  \
0                                                NaN  2018-01-18 00:00:00   
1                                

In [3]:
df_reviews = df_orders.drop(columns=['order_id','review_id','review_creation_date','review_answer_timestamp'])
df_reviews.head()

,review_score,review_comment_title,review_comment_message
0,4,NaN,NaN
1,5,NaN,NaN
2,5,NaN,NaN
3,5,NaN,Recebi bem antes do prazo estipulado.
4,5,NaN,Parabéns lojas lannister adorei comprar pela I...


In [4]:
df_reviews.shape

(99224, 3)

In [5]:
df_reviews_limpo = df_reviews.dropna(
    subset=['review_comment_title', 'review_comment_message'],
    how='all'
).rename(columns={
    'review_score': 'score',
    'review_comment_title': 'title',
    'review_comment_message': 'comment'
})
df_reviews_limpo["score"] = df_reviews_limpo["score"].map({
    1: 1,
    2: 2,
    3: 2,
    4: 2,
    5: 3
})

print(f"Dataset: {df_reviews_limpo.shape[0]} linhas")
df_reviews_limpo.head()

Dataset: 42706 linhas


,score,title,comment
3,3,NaN,Recebi bem antes do prazo estipulado.
4,3,NaN,Parabéns lojas lannister adorei comprar pela I...
9,2,recomendo,aparelho eficiente. no site a marca do aparelh...
12,2,NaN,"Mas um pouco ,travando...pelo valor ta Boa.\r\n"
15,3,Super recomendo,"Vendedor confiável, produto ok e entrega antes..."


## EDA

##TF-IDF

In [31]:
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

# --- Carregar dados ---
df = df_reviews_limpo

# --- Unir title + comment ---
df["texto"] = (
    df["title"].fillna("") + " " + df["comment"].fillna("")
).str.strip()

df = df[df["texto"] != ""]

X = df["texto"]
y = df["score"]  # 1 a 5

# --- Split ---
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# --- Pipeline ---
modelo = Pipeline([
    ("tfidf", TfidfVectorizer(
        ngram_range=(1, 2),
        max_df=0.9,
        min_df=2
    )),
    ("clf", LogisticRegression(
        max_iter=1000,
        multi_class="multinomial"
    ))
])

# --- Treinar ---
modelo.fit(X_train, y_train)

# --- Predições ---
y_pred = modelo.predict(X_test)

# --- DataFrame de análise ---
resultado = pd.DataFrame({
    "real": y_test.values,
    "predito": y_pred
})

# --- Estatísticas por classe ---
print("CLASSE | TOTAL | CERTOS | ERRADOS | ACURÁCIA")
print("-" * 45)

for nota in [1, 2, 3]:
    df_nota = resultado[resultado["predito"] == nota]

    total = len(df_nota)
    certos = (df_nota["real"] == nota).sum()
    errados = total - certos

    acuracia = certos / total if total > 0 else 0

    print(f"{nota:^6} | {total:^5} | {certos:^6} | {errados:^7} | {acuracia:.2%}")

# --- Acurácia geral ---
acc_geral = accuracy_score(y_test, y_pred)

y_train_pred = modelo.predict(X_train)
y_test_pred = modelo.predict(X_test)
acc_train = accuracy_score(y_train, y_train_pred)
acc_test = accuracy_score(y_test, y_test_pred)

#salva modelo
nome_arquivo = "modelo_sentimental_final.pkl"
joblib.dump(modelo, nome_arquivo)

print(f"Acurácia TREINO: {acc_train:.3f}")
print(f"Acurácia TESTE : {acc_test:.3f}")

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


CLASSE | TOTAL | CERTOS | ERRADOS | ACURÁCIA
---------------------------------------------
  1    | 1890  |  1381  |   509   | 73.07%
  2    | 1755  |  1074  |   681   | 61.20%
  3    | 4893  |  3932  |   961   | 80.36%
Acurácia TREINO: 0.829
Acurácia TESTE : 0.748


## database do algoritmo de atrasos

In [7]:
file_to_load = os.path.join(extracted_folder, 'olist_orders_dataset.csv')
df_data = pd.read_csv(file_to_load)
print(df_orders.head())

                          review_id                          order_id  \
0  7bc2406110b926393aa56f80a40eba40  73fc7af87114b39712e6da79b0a377eb   
1  80e641a11e56f04c1ad469d5645fdfde  a548910a1c6147796b98fdf73dbeba33   
2  228ce5500dc1d8e020d8d1322874b6f0  f9e4b658b201a9f2ecdecbb34bed034b   
3  e64fb393e7b32834bb789ff8bb30750e  658677c97b385a9be170737859d3511b   
4  f7c4243c7fe1938f181bec41a392bdeb  8e6bfb81e283fa7e4f11123a3fb894f1   

   review_score review_comment_title  \
0             4                  NaN   
1             5                  NaN   
2             5                  NaN   
3             5                  NaN   
4             5                  NaN   

                              review_comment_message review_creation_date  \
0                                                NaN  2018-01-18 00:00:00   
1                                                NaN  2018-03-10 00:00:00   
2                                                NaN  2018-02-17 00:00:00   
3           

In [19]:
df_unido = pd.merge(df_data, df_orders, on='order_id', how='inner')
df_unido.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,a54f0611adc9ed256b57ede6b6eb5114,4,NaN,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,8d5266042046a06655c8db133d120ba5,4,Muito boa a loja,Muito bom o produto.,2018-08-08 00:00:00,2018-08-08 18:37:50
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,e73b67b67587f7644d5bd1a52deb1b01,5,NaN,NaN,2018-08-18 00:00:00,2018-08-22 19:07:58
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,359d03e676b3c069f62cadba8dd3f6e8,5,NaN,O produto foi exatamente o que eu esperava e e...,2017-12-03 00:00:00,2017-12-05 19:21:58
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,e50934924e227544ba8246aeb3770dd4,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 13:02:51


In [20]:
dfA=df_unido.drop(columns=['order_id','customer_id','order_status','order_purchase_timestamp','order_approved_at','order_delivered_carrier_date','review_answer_timestamp','review_creation_date','review_id'])
dfA=dfA.dropna(subset=['review_comment_title', 'review_comment_message'],how='all')
dfA.head()

,order_delivered_customer_date,order_estimated_delivery_date,review_score,review_comment_title,review_comment_message
0,2017-10-10 21:25:13,2017-10-18 00:00:00,4,NaN,"Não testei o produto ainda, mas ele veio corre..."
1,2018-08-07 15:27:45,2018-08-13 00:00:00,4,Muito boa a loja,Muito bom o produto.
3,2017-12-02 00:28:42,2017-12-15 00:00:00,5,NaN,O produto foi exatamente o que eu esperava e e...
6,NaN,2017-05-09 00:00:00,2,NaN,fiquei triste por n ter me atendido.
10,2017-05-29 11:18:31,2017-06-07 00:00:00,1,NaN,Aguardando retorno da loja


In [17]:
dfA.shape

(42706, 5)

In [22]:
dfA['order_delivered_customer_date'] = pd.to_datetime(dfA['order_delivered_customer_date'])
dfA['order_estimated_delivery_date'] = pd.to_datetime(dfA['order_estimated_delivery_date'])
dfA['atrasado'] = ((dfA['order_delivered_customer_date'] > dfA['order_estimated_delivery_date']) | (dfA['order_delivered_customer_date'].isna())).astype(int)
dfA.head()

,order_delivered_customer_date,order_estimated_delivery_date,review_score,review_comment_title,review_comment_message,atrasado
0,2017-10-10 21:25:13,2017-10-18,4,NaN,"Não testei o produto ainda, mas ele veio corre...",0
1,2018-08-07 15:27:45,2018-08-13,4,Muito boa a loja,Muito bom o produto.,0
3,2017-12-02 00:28:42,2017-12-15,5,NaN,O produto foi exatamente o que eu esperava e e...,0
6,NaT,2017-05-09,2,NaN,fiquei triste por n ter me atendido.,1
10,2017-05-29 11:18:31,2017-06-07,1,NaN,Aguardando retorno da loja,0


In [24]:
dfA["texto"] = (dfA["review_comment_title"].fillna("") + " " + dfA["review_comment_message"].fillna("")).str.strip()
dfA=dfA.drop(columns=["review_comment_title","review_comment_message",'order_estimated_delivery_date','order_delivered_customer_date'])
dfA.head()

,review_score,atrasado,texto
0,4,0,"Não testei o produto ainda, mas ele veio corre..."
1,4,0,Muito boa a loja Muito bom o produto.
3,5,0,O produto foi exatamente o que eu esperava e e...
6,2,1,fiquei triste por n ter me atendido.
10,1,0,Aguardando retorno da loja


In [25]:
dfA.groupby('atrasado').count()

,review_score,texto
atrasado,,
0,36418,36418
1,6288,6288


##modelo de prediçãod e atrasos

In [29]:
XA = dfA["texto"]
yA = dfA["atrasado"]  # 1 a 5

# --- Split ---
X_train, X_test, y_train, y_test = train_test_split(
    XA, yA, test_size=0.2, random_state=42, stratify=yA
)

# --- Pipeline ---
modelo = Pipeline([
    ("tfidf", TfidfVectorizer(
        ngram_range=(1, 2),
        max_df=0.9,
        min_df=2
    )),
    ("clf", LogisticRegression(
        max_iter=1000,
    ))
])

# --- Treinar ---
modelo.fit(X_train, y_train)

# --- Predições ---
y_pred = modelo.predict(X_test)

# --- DataFrame de análise ---
resultado = pd.DataFrame({
    "real": y_test.values,
    "predito": y_pred
})

# --- Estatísticas por classe ---
print("CLASSE | TOTAL | CERTOS | ERRADOS | ACURÁCIA")
print("-" * 45)

for nota in [0, 1]: # Changed to reflect binary classes (0: not delayed, 1: delayed)
    df_nota = resultado[resultado["predito"] == nota]

    total = len(df_nota)
    certos = (df_nota["real"] == nota).sum()
    errados = total - certos

    acuracia = certos / total if total > 0 else 0

    print(f"{nota:^6} | {total:^5} | {certos:^6} | {errados:^7} | {acuracia:.2%}")

# --- Acurácia geral ---
acc_geral = accuracy_score(y_test, y_pred)

y_train_pred = modelo.predict(X_train)
y_test_pred = modelo.predict(X_test)
acc_train = accuracy_score(y_train, y_train_pred)
acc_test = accuracy_score(y_test, y_test_pred)


#salva modelo
nome_arquivo = "modelo_atraso_final.pkl"
joblib.dump(modelo, nome_arquivo)


print(f"Acurácia TREINO: {acc_train:.3f}")
print(f"Acurácia TESTE : {acc_test:.3f}")

CLASSE | TOTAL | CERTOS | ERRADOS | ACURÁCIA
---------------------------------------------
  0    | 7743  |  7140  |   603   | 92.21%
  1    |  799  |  655   |   144   | 81.98%
Acurácia TREINO: 0.927
Acurácia TESTE : 0.913
